#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [2]:
import os, sys 

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

# from src import __version__
# print(__version__)

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [3]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [4]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1 * 365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [5]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())

# if repo_root == os.getcwd(): #then go one directory above to reach repo root
#     repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if "d:\\Work" in repo_root or "D:\\Work" in repo_root:
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [9]:

import importlib
import get_stocks_data as gs
importlib.reload(gs)

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["ESSARSHPNG.NS","LATENTVIEW.NS","BHEL.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2026-02-03 12:18:42,313 - INFO - 📈 Fetching data for 2178 tickers...
2026-02-03 12:18:42,313 - INFO - Fetching data from 2025-01-04 to 2026-02-03


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*                      2%                       ]  37 of 2178 completed2026-02-03 12:18:46,782 - ERROR - $SMLISUZU.NS: possibly delisted; no price data found  (1d 2025-01-04 12:18:42.313584 -> 2026-02-03 12:18:42.313584) (Yahoo error = "No data found, symbol may be delisted")
[***                    7%                       ]  160 of 2178 completed2026-02-03 12:18:59,762 - ERROR - $ACLGATI.NS: possibly delisted; no price data found  (1d 2025-01-04 12:18:42.313584 -> 2026-02-03 12:18:42.313584) (Yahoo error = "No data found, symbol may be delisted")
[************          24%                       ]  532 of 2178 completed2026-02-03 12:19:37,079 - ERROR - $DHANI.NS: possibly delisted; no price data found  (1d 2025-01-04 12:18:42.313584 -> 2026-02-03 12:18:42.313584) (Yahoo error = "No data found, symbol may be delisted")
[***************       32%                       ]  692 of 2178 completed2026-02-03 12:19:52,821 - ERROR - $SUNDARMHLD.NS: possibly delisted; no price data found  (1d 2

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2025-01-06,20MICRONS.NS,NaN,246.855148,261.817573,246.855148,259.826565,122720.0,246.855148,246.855148,246.855148,246.855148,NaN,NaN,20MICRONS
1,2025-01-07,20MICRONS.NS,NaN,249.005447,253.753991,238.174358,240.911994,83202.0,247.059939,246.939474,246.897728,246.876544,NaN,NaN,20MICRONS
2,2025-01-08,20MICRONS.NS,NaN,244.366394,249.373778,238.920985,249.373778,41548.0,246.803411,246.838569,246.847603,246.851568,NaN,NaN,20MICRONS
3,2025-01-09,20MICRONS.NS,NaN,239.488419,247.681413,235.436710,243.888545,27429.0,246.106745,246.550328,246.701877,246.778303,NaN,NaN,20MICRONS
4,2025-01-10,20MICRONS.NS,NaN,234.222198,240.911987,229.961442,239.488424,40503.0,244.974883,246.066871,246.454754,246.653366,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [10]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

✅ Database updated successfully


,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv


#### EMA Signal indicator 


In [11]:
import importlib
import ema_signal as emasignal
importlib.reload(emasignal)

# Detect Golden/Death cross with volume confirmation

long_stocks, short_stocks, ema_df = emasignal.generate_ema_signals(
    stock_df,
    analysis_period=15,
    vol_thresh=1.5,
    output_period=30
)
long_stocks.head()



status  = emasignal.save_EMA_signal_to_db(long_stocks, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
ema_df.head()

117 tickers with Close/200ema crossover between 2026-01-19–2026-02-03
42 tickers with 50ema/200ema crossover between 2026-01-19–2026-02-03
✅ Metadata fetched for 156 total stocks
✅ Final signals: 78 long | 83 short
✅ Saved 78 new EMA signal records to SIGNAL_EMA_CROSS
📦 Table now has 4169 total records.
✅ Database updated successfully


,Symbol,signal_1,Date_1,Price_1,7dratio_1,30dratio_1,Vol1,Mom1,signal_2,Date_2,Price_2,7dratio_2,30dratio_2,Vol2,Mom2,currentPrice,Capitalization,MCap,upFrom52wlow,PE,PB,PBXPE,debtToEquity,PS,PricetoCash,EPS,CashperShare,update date,beta,AD,AD_50
0,360ONE,Price_gt_200ema,2026-01-23,1112.0,0.66,2.05,2.05,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1171.0,Large Cap,47517.0,0.79,41.0,5.0,208.0,1.47,11.24,37.075379,29.0,32.0,2026-02-03,0.26,NaN,NaN
1,360ONE,Price_lt_200ema,2026-01-21,1095.0,0.39,2.08,2.08,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1171.0,Large Cap,47517.0,0.79,41.0,5.0,208.0,1.47,11.24,37.075379,29.0,32.0,2026-02-03,0.26,NaN,NaN
2,AADHARHFC,Price_gt_200ema,2026-01-30,491.0,2.24,1.24,2.24,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,473.0,Large Cap,20514.0,0.60,20.0,3.0,65.0,2.55,10.02,11.410578,23.0,41.0,2026-02-03,0.00,NaN,NaN
3,AARTIIND,Price_gt_200ema,2026-02-03,427.0,4.80,2.40,4.80,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426.0,Mid Cap,15468.0,0.56,41.0,3.0,113.0,0.70,2.04,91.799569,10.0,5.0,2026-02-03,0.38,NaN,NaN
5,ABB,Price_gt_200ema,2026-01-29,5474.0,4.24,2.27,4.24,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5748.0,Large Cap,121801.0,0.68,69.0,17.0,1163.0,0.01,9.36,23.137991,84.0,248.0,2026-02-03,0.47,NaN,NaN


##### Update the Stock returns table

In [12]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")


✅ Total signals loaded: 2280
✅ Total signals after merging: 369101
📦 Existing SIGNAL_RETURNS records: 2791
✅ New unique records found: 22
💾 SIGNAL_RETURNS table updated successfully.
🔒 Database connection closed.
✅ Signal returns generation successful!


,Symbol,Signal_Type,Signal_date,Signal_Price,Signal_Rank,ret_1w,ret_1w_price,ret_1w_date,ret_2w,ret_2w_price,ret_2w_date,ret_1m,ret_1m_price,ret_1m_date,ret_3m,ret_3m_price,ret_3m_date,ret_6m,ret_6m_price,ret_6m_date,ret_1y,ret_1y_price,ret_1y_date,ret_sinceSignal,current_price,current_date,ret_1w_perc,ret_2w_perc,ret_1m_perc,ret_3m_perc,ret_6m_perc,ret_1y_perc,ret_sinceSignal_perc
189,ASTRAMICRO,Golden cross,2026-01-28,962.0,3,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-1.62,946.450012,2026-02-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.62
295,BIGBLOC,Golden cross,2026-01-16,70.0,2,-17.59,57.689999,2026-01-23,-18.39,57.130001,2026-01-30,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-19.43,56.400002,2026-02-02,-17.59,-18.39,<NA>,<NA>,<NA>,<NA>,-19.43
473,DATAPATTNS,Death cross,2026-01-27,2297.0,1,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,11.35,2557.600098,2026-02-02,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,11.35
482,DBOL,Golden cross,2026-02-02,102.0,1,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-1.60,100.370003,2026-02-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.60
483,DBOL,Golden cross,2026-02-01,102.0,1,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaT,-1.60,100.370003,2026-02-03,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-1.60
